# VAMP

For users already familiar with the VAMP interface: The corresponding [API docs](../api/generated/sktime.decomposition.VAMP.rst#sktime.decomposition.VAMP).

VAMP is short for [V]ariational [A]pproach for [M]arkov [P]rocesses and was introduced in <cite data-cite="wu2020variational">(Wu, 2020)</cite>. It can be used as linear dimensionality reduction tool and extends [TICA](tica.ipynb) in the sense that it gives rise to a systematic way to select input features as well as handles off-equilibrium data consistently.

While the VAMP model possesses methods to project data into lower dimensions, it also is equipped with scoring functions (the so-called VAMP scores), which allow ranking of features. This has been demonstrated in, e.g., <cite data-cite="scherer2019variational">(Scherer, 2019)</cite>.